# Import all required libraries

In [3]:
import pandas as pd
import numpy as np
import torch
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.nn import PyroModule, PyroParam

# Read all data into Pandas DataFrames

In [4]:
def process_feature_file(file_path: str, feature_name: str) -> pd.DataFrame:
    """
    Process a single feature file into a DataFrame row with month_year columns.
    
    Args:
        file_path: Path to the CSV file.
        feature_name: Name of the feature (used as the row index).
    
    Returns:
        DataFrame with 1 row and columns like 'jan_1999', 'feb_1999', etc.
    """
    df = pd.read_csv(file_path)
    feature_data = pd.DataFrame()

    for _, row in df.iterrows():
        year = row['Year']
        monthly_values = row.drop('Year')
        month_year_columns = {month: f"{month.lower()}_{year}" 
                             for month in monthly_values.index}
        
        # Create single-row DataFrame for this year's data
        year_df = pd.DataFrame([monthly_values.values], 
                              columns=month_year_columns.values())
        feature_data = pd.concat([feature_data, year_df], axis=1)

    feature_data.index = [feature_name]
    return feature_data

# List of tuples (file_path, feature_name)
DATA_PATH = '../data/wales/'
files_to_process = [
    (DATA_PATH+'wales_airfrost_days.csv', 'airfrost_days'),
    (DATA_PATH+'wales_precipitation_per_month.csv', 'rainfall'), 
    (DATA_PATH+'wales_monthly_temp.csv', 'temperature'),
    (DATA_PATH+'wales_monthly_sunshine.csv', 'sunshine')
]

# Process all files and combine
dataframes = []
for file_path, feature in files_to_process:
    df = process_feature_file(file_path, feature)
    dataframes.append(df)

final_df = pd.concat(dataframes)

print(final_df)

               jan_1999.0  feb_1999.0  mar_1999.0  apr_1999.0  may_1999.0  \
airfrost_days         6.6         6.1         3.2         4.3         0.0   
rainfall            250.9       104.0       100.2       122.5        83.4   
temperature           NaN         NaN         6.4         8.5        11.8   
sunshine             43.9        53.7        91.9       144.8       131.9   

               june_1999.0  july_1999.0  aug_1999.0  sep_1999.0  oct_1999.0  \
airfrost_days          0.1          0.0         0.0         0.0         0.4   
rainfall               NaN          NaN       138.0       204.3       137.4   
temperature            NaN          NaN        15.1        14.0        10.0   
sunshine               NaN          NaN       144.9         NaN       109.1   

               ...  sept_2015.0  sept_2016.0  sept_2017.0  sept_2018.0  \
airfrost_days  ...          NaN          NaN          NaN          NaN   
rainfall       ...          NaN          NaN          NaN          NaN